In [6]:
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
# TODO: Change Models You want https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet
from tensorflow.keras.applications.vgg19 import VGG19
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import numpy as np

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.1


## **CIFAR 10 Datast 데이터셋 다운로드하고 준비하기**




In [10]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  # TODO: Change Models You want to change
  output_ims = tf.keras.applications.vgg19.preprocess_input(input_images)
  return output_ims

In [11]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

In [12]:
train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [13]:
train_X.shape

(50000, 32, 32, 3)

In [14]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  #Change Models Here too!!!
  
  feature_extractor = tf.keras.applications.VGG19(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    vgg19_feature_extractor = feature_extractor(resize)
    classification_output = classifier(vgg19_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='Adam', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_______________________________________________________

In [15]:
EPOCHS = 3
history = model.fit(train_X, training_labels, epochs=EPOCHS, validation_data = (valid_X, validation_labels), batch_size=64)

Epoch 1/3
782/782 [==============================] - 374s 478ms/step - loss: 2.0454 - accuracy: 0.2742 - val_loss: 1.6945 - val_accuracy: 0.3696
Epoch 2/3
782/782 [==============================] - 373s 477ms/step - loss: 1.5920 - accuracy: 0.4149 - val_loss: 1.5073 - val_accuracy: 0.4505
Epoch 3/3
782/782 [==============================] - 373s 477ms/step - loss: 1.4048 - accuracy: 0.4890 - val_loss: 1.3763 - val_accuracy: 0.5073


In [16]:
loss, accuracy = model.evaluate(valid_X, validation_labels, batch_size=64)

157/157 [==============================] - 21s 131ms/step - loss: 1.3763 - accuracy: 0.5073
